# Getting Physical Compute Inventory from Intersight using the Cisco Intersight Python SDK

In this lab you learn how to retrieve a list of physical compute inventory from Cisco Intersight using the Intersigight Python SDK.

## Objectives

The objective of this lab is to show how to:

* Authenticate with the Intersight using the Python SDK API's
* Call the Intersight API to pull a list of physical compute resources

## Prerequisites

Before getting started with this lab, you will need the following:

* An Intersight account (you may also need a Cisco account for credentials)
* API key
* API secret available
* Familiarity with Python

### Background

Intersight has many capabilities and one of them is managing on-premises compute resources such as Cisco UCS servers. The process of making UCS servers available to Intersight is known as a claim. Thus, an administrator must first claim UCS servers before they are available as a resource in Intersight. After a UCS servers are claimed, they show up in inventory as physical compute servers.

In this lab we call the Intersight REST API's using Python and retrieve the inventory of physical compute resources claimed in Intersight.

## Step 1: Generate API key and Secret Key

Log into your Intersight account and navigate to **Settings** which is available by clicking the gear icon located towaward the upper-righthand side of the Intersight user interface.

<img align="left" src="images/intersight-settings.png">

* Click **API Keys** located in the lefthand column.
* Click **Generate API Key** located toward the upper-righthand side of the page.
* Select **API key for OpenAPI schema version 2** in the **Generate API Key** dialog box and provide a description for your API key.
* Copy the API Key ID and store it somewhere for use in the upcoming steps.
* Save the **Secret Key** to a text file and make note of its location. Move the file to the same directory where this code is running or make note of its location for use in the upcoming steps.

<img align="left" src="images/api-secret-key-save-as.png">

> This is the only one time that the secret key can be viewed or downloaded. You cannot recover them later. However, you can create new access keys at any time.

## Step 2: Install the Intersight Python SDK and import modules

The Python Intersight SDK is available in the [Python Packaging Index](https://pypi.org/)  and installable using the command `pip install intersight`.

> Be sure to running Python >= 3.6 as earlier versions of Python are not supported. Also, uninstall any conflicting versions of the SDK installed on your machine. You can check installed versions with the `pip list` command prior to running `pip install intersight`.

In [1]:
pip install intersight

Note: you may need to restart the kernel to use updated packages.


Running the command `pip list` all packages installed by `pip`. Run the command and scroll down to verify the `intersight` pack is installed.

In [12]:
pip list

Package                            Version
---------------------------------- -------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.0
appscript                          1.1.1
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              1.1.1
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autoflake                          1.4
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.

ucsmsdk                            0.9.10
ujson                              4.0.1
unicodecsv                         0.14.1
urllib3                            1.25.11
watchdog                           0.10.3
wcwidth                            0.2.5
webencodings                       0.5.1
Werkzeug                           1.0.1
wheel                              0.35.1
widgetsnbextension                 3.5.1
wrapt                              1.11.2
wurlitzer                          2.0.1
xlrd                               1.2.0
XlsxWriter                         1.3.7
xlwings                            0.20.8
xlwt                               1.3.0
xmltodict                          0.12.0
yapf                               0.30.0
zict                               2.0.0
zipp                               3.4.0
zope.event                         4.5.0
zope.interface                     5.1.2
Note: you may need to restart the kernel to use updated packages.


Now that the `intersight` package is installed using `pip` you are ready to import modules needed putting this example together. Start by importing `intersight`, `datetime`, and `timedelta`.

In [182]:
import intersight
import intersight.api.compute_api

 > Interested in learning more about the `intersight` module? If so, use the command `help(intersight)` to see a description of the module as shown below.

In [183]:
help(intersight)

Help on package intersight:

NAME
    intersight - Cisco Intersight

DESCRIPTION
    Cisco Intersight is a management platform delivered as a service with embedded analytics for your Cisco and 3rd party IT infrastructure. This platform offers an intelligent level of management that enables IT organizations to analyze, simplify, and automate their environments in more advanced ways than the prior generations of tools. Cisco Intersight provides an integrated and intuitive management experience for resources in the traditional data center as well as at the edge. With flexible deployment options to address complex security needs, getting started with Intersight is quick and easy. Cisco Intersight has deep integration with Cisco UCS and HyperFlex systems allowing for remote deployment, configuration, and ongoing maintenance. The model-based deployment works for a single system in a remote location or hundreds of systems in a data center and enables rapid, standardized configuration and depl

## Step 3: Configuring and Creating the API Client

In step 1, you retrieved your API key and secret key and in this step you apply those values while configuring a newly created API client. The command `help(intersight.Configuration)` displays information about the `Configuration` Python class and its usage.

In [184]:
help(intersight.Configuration)

Help on class Configuration in module intersight.configuration:

class Configuration(builtins.object)
 |  Configuration(host=None, api_key=None, api_key_prefix=None, access_token=None, username=None, password=None, discard_unknown_keys=False, disabled_client_side_validations='', signing_info=None, server_index=None, server_variables=None, server_operation_index=None, server_operation_variables=None, ssl_ca_cert=None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator
 |  
 |      Ref: https://openapi-generator.tech
 |      Do not edit the class manually.
 |  
 |      :param host: Base url
 |      :param api_key: Dict to store API key(s).
 |        Each entry in the dict specifies an API key.
 |        The dict key is the name of the security scheme in the OAS specification.
 |        The dict value is the API key secret.
 |      :param api_key_prefix: Dict to store API prefix (e.g. Bearer)
 |        The dict key is the name of the security scheme in the OAS specification.

Help is also available for the `ApiClient` class with the command `help(intersight.ApiClient)`

In [185]:
help(intersight.ApiClient)

Help on class ApiClient in module intersight.api_client:

class ApiClient(builtins.object)
 |  ApiClient(configuration=None, header_name=None, header_value=None, cookie=None, pool_threads=1)
 |  
 |  Generic API client for OpenAPI client library builds.
 |  
 |  OpenAPI generic API client. This client handles the client-
 |  server communication, and is invariant across implementations. Specifics of
 |  the methods and models for each application are generated from the OpenAPI
 |  templates.
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator.
 |  Ref: https://openapi-generator.tech
 |  Do not edit the class manually.
 |  
 |  :param configuration: .Configuration object for this client
 |  :param header_name: a header to pass when making calls to the API.
 |  :param header_value: a header value to pass when making calls to
 |      the API.
 |  :param cookie: a cookie to include in the header when making calls
 |      to the API
 |  :param pool_threads: The number of thread

#### Configure the API Client

Now that `help` provided you with, albeit a lot of background information, you are ready to configurate and create the API client as shown in the example below.

> Keep in mind that the values below are for demonstration purposes only and the values of the variables will not work in your environment or the example below. You use the values of the secret key and API key derived in Step 1. `API Key ID` maps to `key_id` and `Secret Key` maps to `private_key_path` in the example below.

In [186]:
configuration = intersight.Configuration(
    signing_info=intersight.HttpSigningConfiguration(
        key_id='insert-your-api-key-value-here',
        private_key_path='/Users/delgadm/Documents/intersight/intersight-jupyter-notebooks/key/some-secret-key.txt',
        signing_scheme=intersight.signing.SCHEME_HS2019,
        signed_headers=[intersight.signing.HEADER_HOST,
                        intersight.signing.HEADER_DATE,
                        intersight.signing.HEADER_DIGEST,
                        intersight.signing.HEADER_REQUEST_TARGET
                        ]
    )
)

#### Create the API Client

Next, create the client and store the resulting handle in the a variable named `api_client` by calling the `Api.Client` class and passing the values stores in `configuration` to it as shown below.

In [187]:
api_client = intersight.ApiClient(configuration)

## Step 4: Query the API for Physical Compute Inventory

Now that you are authenticated, let's use the SDK to query for the number of physical compute items in inventory. You first pass `api_client` variable defined in the previous step to `intersight.api.compute_api.ComputeApi` and store the result in a new variable named `api_instance` as shown below.

In [188]:
# Get compute class instance
api_instance = intersight.api.compute_api.ComputeApi(api_client)

#### Make the Query and Add a Filter that Retrieves Only UCS X-Series

In the next line of code, you will query the API with a filter so that we only ask for UCS X-Series physical compute nodes and store the returned value in a variable named `compute_inventory`. If you remove the `filter='contains(Model,\'UCSX\')'` argument, everything in inventory is returned.

In [189]:
compute_inventory = api_instance.get_compute_physical_summary_list(filter='contains(Model,\'UCSX\')')

## Step 5: See How Many Physical Compute Items Were Returned

The object returned by the API call contains a list of attributes and methods you see when using the `dir()` function.

In [190]:
dir(compute_inventory)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unset_attribute_value__',
 '__weakref__',
 '_additional_properties_model_instances',
 '_check_type',
 '_composed_instances',
 '_composed_schemas',
 '_configuration',
 '_data_store',
 '_nullable',
 '_path_to_item',
 '_spec_property_naming',
 '_var_name_to_model_instances',
 '_visited_composed_classes',
 'additional_properties_type',
 'allowed_values',
 'attribute_map',
 'discriminator',
 'get',
 'openapi_types',
 'required_properties',
 'set_attribute',
 'to_dict',
 'to_str',
 'validations']

One of the attributes you see is a list named `results`. If you pass that to the `len()` BIF (built in function), it returns the number of items in the list which tells you the number of UCS X-Series in inventory.

In [191]:
len(compute_inventory.results)

6

#### Show Everything Returned by the Query

Printing `compute_inventory.results` shows a rich set of information returned about physical compute items in inventory.

In [192]:
compute_inventory.results

[{'account_moid': '596cc79e5d91b400010d15ad',
  'admin_power_state': '',
  'alarm_summary': {'class_id': 'compute.AlarmSummary',
                    'critical': 4,
                    'object_type': 'compute.AlarmSummary',
                    'warning': 0},
  'ancestors': [{'class_id': 'mo.MoRef',
                 'link': 'https://www.intersight.com/api/v1/equipment/Chasses/60e228f66176752d37a47da8',
                 'moid': '60e228f66176752d37a47da8',
                 'object_type': 'equipment.Chassis'}],
  'asset_tag': '',
  'available_memory': 512,
  'bios_post_complete': True,
  'chassis_id': '1',
  'class_id': 'compute.PhysicalSummary',
  'connection_status': '',
  'cpu_capacity': 140.8,
  'create_time': datetime.datetime(2021, 7, 4, 21, 35, 4, 223000, tzinfo=tzutc()),
  'device_mo_id': '60e229446f72612d33cafe2f',
  'dn': '/redfish/v1/Systems/FCH243974X1',
  'domain_group_moid': '5b2541877a7662743465cc98',
  'fault_summary': 0,
  'firmware': '5.0(1b)',
  'inventory_device_info': N

## Step 6: Bringing it all together
Great! Now we see the number of physical compute devices and now we can pull more information from the returned JSON and organize it by Device, Chassis ID, Management Mode, Model, Memory, and CPU. The information in the `Management Mode` column shows if the phsyical compute device is managed in Intersight Management Mode.

> Intersight Managed Mode (IMM) is a new architecture that manages the UCS Fabric Interconnected systems through a Redfish-based standard model. If you are familiar with the UCS blades, it means the Fabric Interconnect is fully managed by Intersight. Instead of having the familiar UCSM (UCS Manager) interface available directly from the Fabric Interconnect, the interface and all of the Fabric Interconnect operations are managed by Intersight.

We do some CLI formatting to organize our data and see the type of compute hardware managed by Intersight along with its resources (memory and CPU). Then, we iterate over the JSON data and pull the data we're interested in. In this instance, the Model shows the UCS-X series hardware.

> Experiment! See if you can add more information to the list below by choosing other items from the returned JSON data above. For example, you could add a column with `num_cpu_cores` and or `ipv4_address` to the code below. There's no right or wrong answer as to which columns you would like displayed.

In [193]:
print ("{:<8} {:12} {:<21} {:<15} {:<10} {:<10}".format(
       'Device',
       'Chassis ID',
       'Management Mode',
       'Model',
        'Memory',
        'CPU'))

for num, items in enumerate(compute_inventory.results, start=1):
    print (
        "{:<8} {:<12} {:<21} {:<15} {:<10} {:<10}".format(
            num, 
            items.chassis_id, 
            items.management_mode, 
            items.model, 
            items.available_memory, 
            items.cpu_capacity))

Device   Chassis ID   Management Mode       Model           Memory     CPU       
1        1            Intersight            UCSX-210C-M6    512        140.8     
2        1            Intersight            UCSX-210C-M6    1152       145.59999 
3        1            Intersight            UCSX-210C-M6    512        145.59999 
4        1            Intersight            UCSX-210C-M6    512        108.0     
5        1            Intersight            UCSX-210C-M6    512        145.59999 
6        1            Intersight            UCSX-210C-M6    320        145.59999 
